In [2]:
%load_ext autoreload

%autoreload 2

from vae_dist.core.VAE import baselineVAEAutoencoder
from vae_dist.core.training import train
from vae_dist.dataset.dataset import FieldDataset
import torch 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pytorch_lightning'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root = "../../data/cpet/"
# load model to gpu
dataset_vanilla = FieldDataset(
    root, 
    transform=None, 
    augmentation=None, 
    device=device
    )


dataset_loader_full = torch.utils.data.DataLoader(dataset_vanilla, batch_size=40, shuffle=True)


In [ ]:
model = baselineVAEAutoencoder(
    irreps = None, # not used rn 
    in_channels = 3,
    out_channels = 16,
    kernel_size = 5,
    stride = 1,
    padding = 0,
    dilation = 1,
    groups = 1,
    bias = True,
    padding_mode = 'zeros',
    #padding_mode = 'constant',
    latent_dim = 4, # final vae hidden layer 
    num_layers = 2, # not used rn 
    hidden_dim = 32,
    activation = 'relu', # not used rn 
    dropout = 0.1, # not used rn 
    batch_norm = False, # not used rn 
    beta = 1.0,
    device = device, 
    loss = 'elbo'
)

In [ ]:
train(model, dataset_loader_full, device = device, epochs = 100)

# Lightning Implementation

In [1]:
%load_ext autoreload
%autoreload 2

import pytorch_lightning as pl
from vae_dist.core.CNNLightning import CNNAutoencoderLightning
from vae_dist.dataset.dataset import FieldDataset
from pytorch_lightning.callbacks import LearningRateMonitor
import torch 

/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root = "../../data/cpet/"
# load model to gpu
dataset_vanilla = FieldDataset(
    root, 
    transform=None, 
    augmentation=None, 
    device=device
    )

#torch.multiprocessing.set_start_method('spawn')# good solution !!!!

# train test split - randomly split dataset into train and test
train_size = int(0.8 * len(dataset_vanilla))
test_size = len(dataset_vanilla) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset_vanilla, [train_size, test_size])


dataset_loader_full = torch.utils.data.DataLoader(
    dataset_vanilla, 
    batch_size=40,
    shuffle=True,
    num_workers=0
)

dataset_loader_train= torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=10,
    shuffle=True,
    num_workers=0
)

dataset_loader_test= torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=10,
    shuffle=True,
    num_workers=0
)



In [3]:
model = CNNAutoencoderLightning(
    irreps = None, # not used rn 
    in_channels = 3,
    out_channels = 16,
    kernel_size = 5,
    stride = 1,
    padding = 0,
    dilation = 1,
    groups = 1,
    bias = True,
    padding_mode = 'zeros',
    latent_dim = 4, # final vae hidden layer 
    num_layers = 2, # not used rn 
    hidden_dim = 32,
    activation = 'relu', # not used rn 
    dropout = 0.1, # not used rn 
    batch_norm = False, # not used rn 
    beta = 1.0,
    device = device
)

In [4]:
lr_monitor = LearningRateMonitor(logging_interval='step')
trainer = pl.Trainer(
    limit_train_batches=100, 
    max_epochs=100, 
    accelerator='gpu', 
    devices = [0],
    accumulate_grad_batches=5, 
    callbacks=[
        pl.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose = False),
        lr_monitor]
    )

trainer.fit(model, dataset_loader_train, dataset_loader_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 2.5 M 
1 | decoder | Sequential | 2.6 M 
---------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.486    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:491: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  category=PossibleUserWarning,
/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1599: PossibleUserWarning: The number of training batches (29) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Epoch 99: 100%|██████████| 37/37 [00:00<00:00, 319.31it/s, loss=756, v_num=16]     

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 37/37 [00:00<00:00, 180.97it/s, loss=756, v_num=16]


zsh:1: command not found: tensorboard
